# Descarga de datos desde APIs públicas
---
## Consigna

Iniciar el proceso de limpieza y exploración de datos según el dataset elegido para el proyecto final

---



## API a utilizar: RAWG

RAWG es la base de datos más grande de videojuegos que existe en la actualidad, además de ser un servicio de descubrimiento y recomendaciones de videojuegos. Comparten más de 500.000 datos sobre videojuegos, además de proveer recomendaciones basadas en machine learning.

Esta API puede servir de mucho para el proyecto de data science que se viene desarrollando en entregas anteriores, incluso podría servir para reemplazar la base de datos actual y enriquecerla con nueva data.

A continuación, se presenta una exploración de dicha API y la obtención de la data respectiva.

In [42]:
# Llibrerías a utilizar
import requests
import numpy as np
import pandas as pd
from pandas.core.dtypes.cast import date

In [ ]:
# Clave de API de Rawg
api_key = "00565200b6154b65b4751d6ee239866d"

# Número máximo de juegos a obtener
max_games = 10000

# Lista vacía para almacenar los datos de los juegos
game_data = []

# Inicializar la variable page en 1
page = 1

# Ciclo para obtener los datos de los juegos
while len(game_data) < max_games:
    
    # URL de la API de Rawg para obtener los juegos de la página actual
    url = f"https://api.rawg.io/api/games?key={api_key}&page={page}"
    
    # Solicitud GET a la API de Rawg
    response = requests.get(url)
    
    # Convertir la respuesta JSON en un diccionario de Python
    data = response.json()   
    # Obtener los datos de cada juego de la página actual y añadirlos a la lista
    for result in data["results"]:
        game_data.append({
            "Nombre": result.get("name", ""),
            "Fecha_Lanzamiento": result.get("released", ""),
            "Plataformas": ", ".join([platform["platform"]["name"] for platform in result.get("platforms", [])]),
            "Generos": ", ".join([genre["name"] for genre in result.get("genres", [])]),
            "Desarrolladores": ", ".join([developer["name"] for developer in result.get("developers", [])]),
            "Editores": ", ".join([publisher["name"] for publisher in result.get("publishers", [])]),
            "Puntuacion": result.get("rating", ""),
            "Puntuacion_Top": result.get("rating_top", ""),
            "Descripcion": result.get("description", ""),
            "URL_Imagen": result.get("background_image", ""),
            "Sitio_Web": result.get("website", ""),
            "Metracritic": result.get("metacritic", ""),
            "Rating_ESRB": result.get("esrb_rating")["name"] if result.get("esrb_rating") is not None else "",
            "Rating_PEGI": result.get("pegi_rating", {}).get("name", ""),
            "Tags": ", ".join([tag["name"] for tag in result.get("tags", [])]),
            "Comentarios": result.get("comments_count", ""),
            "Reviews": result.get("reviews_count", ""),
            "Playtime": result.get("playtime", ""),
            "Playtime_Completionist": result.get("playtime_completionist", ""),
            "Median_Playtime": result.get("playtime_median", ""),
        })
    
    # Incrementar la variable page en 1 para obtener la siguiente página
    page += 1
    
# Crear dataframe
df = pd.DataFrame(game_data)

# Mostrar el dataframe
df

In [ ]:
# Data Wrangling

# Contamos valores nulos en cada columna
print(df.isna().sum())

# Eliminamos duplicados en caso de que existan
df = df.drop_duplicates()

# Cambio de tipos de variable Fecha_Lanzamiento
df["Fecha_Lanzamiento"] = pd.to_datetime(df["Fecha_Lanzamiento"], errors='coerce').dt.date

# Eliminamos las filas con valores nulos en Fecha_Lanzamiento
df = df.dropna(subset=['Fecha_Lanzamiento'])

# Rellenamos todas las filas que tengan vallores nulos en URL_Imagen
df['URL_Imagen'] = df['URL_Imagen'].fillna('Not Found')

# Calculamos el valor promedio de la columna Metacritic
mean_metacritic = df["Metracritic"].mean()

# Rellenamos los valores nulos de la columna Metacritic con el valor promedio
df["Metracritic"].fillna(mean_metacritic, inplace=True)

In [40]:
#Exportamos a .csv
df.to_csv("juegos_rawg.csv", index=False)